In [194]:
import csv
import random
import numpy as np
from sklearn import svm

In [171]:
#csv_file = open("filtered_h1b.csv")
csv_file = open("subset_h1b.csv")
#list of lists, each list is the list of the shit
lines = list(csv.reader(csv_file,delimiter=','))
train = lines[:25000]
valid = lines[25000:37500]
test = lines[37500:]
validtest = lines[25000:]

In [224]:
#baseline trivial predictor using Salary
def trivial_features(datum):
    feat = [1]
    #Add Salary
    feat.append(float(datum[5]))
    return feat

In [228]:
#predictor using full time status, salary, year of application, location (w/ state)
def features(datum):
    feat = [1]
    #full time status
    if datum[4] == 'Y':
        feat.append(1)
    else:
        feat.append(0)
    
    #salary
    feat.append(float(datum[5]))
    
    #one-hot year of application (2011-2016)
    year = [0]*(2016-2011+1)
    year[2016 - int(datum[6])] = 1
    feat = feat + year

    #location
    states = states_one_hot(datum)
    feat = feat + states
    
    return feat

In [186]:
def get_state_encoding(x):
    return {
        'ALABAMA': 0,
        'ALASKA': 1,
        'ARIZONA': 2,
        'ARKANSAS': 3,
        'CALIFORNIA': 4,
        'COLORADO': 5,
        'CONNECTICUT': 6,
        'DELAWARE': 7,
        'FLORIDA': 8,
        'GEORGIA': 9,
        'HAWAII': 10,
        'IDAHO': 11,
        'ILLINOIS': 12,
        'INDIANA': 13,
        'IOWA': 14,
        'KANSAS': 15,
        'KENTUCKY': 16,
        'LOUISIANA': 17,
        'MAINE': 18,
        'MARYLAND': 19,
        'MASSACHUSETTS': 20,
        'MICHIGAN': 21,
        'MINNESOTA': 22,
        'MISSISSIPPI': 23,
        'MISSOURI': 24,
        'MONTANA': 25,
        'NEBRASKA': 26,
        'NEVADA': 27,
        'NEW HAMPSHIRE': 28,
        'NEW JERSEY': 29,
        'NEW MEXICO': 30,
        'NEW YORK': 31,
        'NORTH CAROLINA': 32,
        'NORTH DAKOTA': 33,
        'OHIO': 34,
        'OKLAHOMA': 35,
        'OREGON': 36,
        'PENNSYLVANIA': 37,
        'RHODE ISLAND': 38,
        'SOUTH CAROLINA': 39,
        'SOUTH DAKOTA': 40,
        'TENNESSEE': 41,
        'TEXAS': 42,
        'UTAH': 43,
        'VERMONT': 44,
        'VIRGINIA': 45,
        'WASHINGTON': 46,
        'WEST VIRGINIA': 47,
        'WISCONSIN': 48,
        'WYOMING': 49,
        'DISTRICT OF COLUMBIA': 50,
        'PUERTO RICO': 51,
        'NA': 52
    }[x]
 
def states_one_hot(review):
    states = [0] * 53;
    location = [x.strip() for x in review[7].split(',')]
    state = location[1]
    states[get_state_encoding(state)] += 1
    return states

In [229]:
#predictor
def predict(X,Y):
    theta,residuals,rank,s = np.linalg.lstsq(X,Y)
    #print('theta: ',theta)
    #print('normalized residuals', residuals/len(X))
    return theta

In [230]:
def mse(theta,X,y):
    # X is the list of list of features
    # y is the actual values of case statuses
    sum = 0
    for data,status in zip(X[:],y[:]):
        #print('d ',data)
        #print('t ',theta)
        
        prediction = np.dot(theta, data)
        sum += (status-prediction)**2
    mse = sum/len(X)
    print(mse)

In [231]:
Y_train = [1 if d[0] == 'CERTIFIED' else 0 for d in train]
Y_valid = [1 if d[0] == 'CERTIFIED' else 0 for d in valid]
Y_test = [1 if d[0] == 'CERTIFIED' else 0 for d in validtest]

X_trivial_train = [trivial_features(d) for d in train]
X_trivial_test = [trivial_features(d) for d in validtest]

X_train = [features(d) for d in train]
X_valid = [features(d) for d in valid]
X_test = [features(d) for d in validtest]

In [239]:
#MSE for Trivial
theta = predict(X_trivial_train,Y_train)
mse(theta,X_trivial_test,Y_test)

0.110008844017


In [240]:
#normal
theta = predict(X_train,Y_train)
mse(theta,X_valid,Y_valid)

0.110457114297


In [258]:
#Linear Predictor
prediction = np.dot(X_valid,theta)
for p in range(0,len(prediction)):
    prediction[p] = int(round(prediction[p]))
validAcc = [(x == y) for (x,y) in zip(prediction,Y_valid)]
validAcc = sum(validAcc)*1.0/len(validAcc)

In [259]:
validAcc

0.87295999999999996

In [217]:
#SVM
X_train = [features(d) for d in train]
Y_train = [1 if d[0] == 'CERTIFIED' else 0 for d in train]
X_valid = [features(d) for d in valid]
Y_valid = [1 if d[0] == 'CERTIFIED' else 0 for d in valid]

clf = svm.LinearSVC(C = 1)
clf.fit(X_train,Y_train)
prediction = clf.predict(X_valid)
validAcc = [(x == y) for (x,y) in zip(prediction,Y_valid)]
validAcc = sum(validAcc)*1.0/len(validAcc)

In [218]:
validAcc

0.87239999999999995

In [219]:
# X_train = [features(d) for d in train]
# Y_train = [1 if d[0] == 'CERTIFIED' else 0 for d in train]
# X_valid = [features(d) for d in valid]
# Y_valid = [1 if d[0] == 'CERTIFIED' else 0 for d in valid]

# bestAcc = 0
# bestCLF = None
# for c in 0.01, 0.1, 1, 10, 100:
#     clf = svm.LinearSVC(C = c)
#     clf.fit(X_train, Y_train)
#     predictions = [int(x) for x in clf.predict(X_valid)]
#     acc = [(x == y) for (x,y) in zip(predictions, Y_valid)]
#     acc = sum(acc) * 1.0 / len(acc)
#     if acc > bestAcc:
#         bestAcc = acc
#         bestCLF = clf
#     print("C= " + str(c) + ": validation acc = " + str(acc))